<a href="https://colab.research.google.com/github/ML-Engineering-Parth/ECED-Circuits-project-arduino/blob/main/Building_a_NN_(From_scratch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal is to create layers, etc. as classes so that they can be used again


In [ ]:
# Creating the base layer

class Layer:
  def __init__(self):
    self.input = None
    self.output = None

  def forward(self, input):
    # This method takes in the input and gives the output

    pass

  def backward(self, output_gradient, learning_rate):

    # This method takes in the dE/dY and updates the training params, and returns the dE/dX (derivative of the error with respect to the input)

    pass

Each input neuron is connected to each output neuron in the dense layer, where each connection represents a weight

In [ ]:
# Creating the Dense layer

import numpy as np

class Dense(Layer):  # Class inheritance from base layer
  def __init__(self, input_size, output_size):
    self.weights = np.random.randn(output_size, input_size) # The shape is something to be careful about
    self.bias = np.random.randn(output_size, 1)

  def forward(self, input):
    # y = WX + B
    self.input = input
    return np.dot(self.weights, self.input) + self.bias

  def backward(self, output_gradient, learning_rate):
    weights_gradient = np.dot(output_gradient, self.input.T)
    final_return = np.dot(self.weights.T, output_gradient)
    self.weights -= learning_rate*weights_gradient # Gradient Descent
    self.bias -= learning_rate * output_gradient # Change the weights after you are done calculating dE/dX
    return np.dot(self.weights.T, output_gradient)  # The logic will be in my notes

The dE/dX of one layer is the dE/dY of the other and we need dE/dY values with us to optimize the paramaters which is why it is important to comput both the dE/dX and the dE/dY for a layer. The dE/dX for layer 2 just ends up being the dE/dY for layer 1, and now that we have the dE/dY for layer 1, we can optimize its parameters

Creating the activation layer

In [ ]:
class Activation(Layer):
  def __init__(self, activation, activation_prime):
    self.activation = activation
    self.activation_prime = activation_prime

  def forward(self, input):
    self.input = input
    return self.activation(self.input)


  def backward(self, output_gradient, learning_rate):
    return np.multiply(output_gradient, self.activation_prime(self.input))


In [ ]:
# Creating the tanh(x) activation function


class Tanh(Activation):
  def __init__(self):
    tanh = lambda x: np.tanh(x)
    tanh_prime = lambda x: 1-np.tanh(x) ** 2
    super().__init__(tanh, tanh_prime)

Implementing the mean squared error

In [ ]:
def mse(y_true, y_pred):
  return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
  return 2 * (y_pred - y_true) / np.size(y_true)

In [ ]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime, x_train, y_train, epochs = 500, learning_rate = 0.01, verbose = True):
    for e in range(epochs):
        error = 0
        for x, y in zip(x_train, y_train):
            # forward
            output = predict(network, x)

            # error
            error += loss(y, output)

            # backward
            grad = loss_prime(y, output)
            for layer in reversed(network):
                grad = layer.backward(grad, learning_rate)

        error /= len(x_train)
        if verbose:
            print(f"{e + 1}/{epochs}, error={error}")

In [ ]:
X = np.reshape([[0, 0], [0, 1], [1, 0], [1, 1]], (4, 2, 1))
Y = np.reshape([[0], [1], [1], [0]], (4, 1, 1))

network = [
    Dense(2, 3),
    Tanh(),
    Dense(3, 1),
    Tanh()
]

train(network, mse, mse_prime, X, Y, epochs=500, learning_rate=0.1)

1/500, error=0.4479431605865309
2/500, error=0.24421584219418185
3/500, error=0.22921317611306888
4/500, error=0.22048792444710547
5/500, error=0.21381755282736276
6/500, error=0.20844888871692904
7/500, error=0.20395289809635828
8/500, error=0.200027679239266
9/500, error=0.19644712045038587
10/500, error=0.19302795465780936
11/500, error=0.1896093079850238
12/500, error=0.186039984329925
13/500, error=0.18216969382872703
14/500, error=0.17784294450024396
15/500, error=0.17289681277685526
16/500, error=0.1671664477319133
17/500, error=0.16050504183168662
18/500, error=0.1528261160704294
19/500, error=0.14416878096139413
20/500, error=0.13476196569599147
21/500, error=0.12502896396008845
22/500, error=0.11548041013167998
23/500, error=0.10653542007004715
24/500, error=0.09839615658986502
25/500, error=0.09105324482212449
26/500, error=0.08438347351050905
27/500, error=0.07825123054085018
28/500, error=0.07256074681620144
29/500, error=0.06726215693459668
30/500, error=0.062336532623656